<a href="https://colab.research.google.com/github/ArrH/thesis/blob/main/ClassicML_SC_DEAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this file we evaluate the performance of the following strategy: 

1. We prepare and normalize the raw GSR signal from the DEAP dataset to be representative of the wearable device acquired Skin Conductivity signal.
2. We get the phasic and tonic components of each signal.
3. We process the labels.
4. We extract features from the signal.
5. We select features related to labels.
6. We test a bunch of algos on these and store the results. We use 10 fold cross validation to assess the model performance with better statistical significance. 


Let's start by extracting relevant data from the raw data files. 

Whe check if the files exists, if not, we extract and save for the use in the future runs of this code. 

In [ ]:
try:
   import cPickle as pickle
except:
   import pickle
import os
import numpy as np

subject_count = 32
trial_count_per_subject = 40
sc_index = 36
raw_data_sample_rate = 128
signal_length = 8064


def get_raw_data(DEAP_data_directory):
  if os.path.isfile(DEAP_data_directory + '/sc_array_raw'):
    print('Raw data exists numpy stored format already. Using that.')
    with open(DEAP_data_directory + '/sc_array_raw', 'rb') as f:
      sc_array_raw = np.array(pickle.load(f))
    with open(DEAP_data_directory + '/valence_array_raw', 'rb') as f:
      valence_array_raw = np.array(pickle.load(f))
    with open(DEAP_data_directory + '/arousal_array_raw', 'rb') as f:
      arousal_array_raw = np.array(pickle.load(f))
    with open(DEAP_data_directory + '/dominance_array_raw', 'rb') as f:
      dominance_array_raw = np.array(pickle.load(f))
    return sc_array_raw, valence_array_raw, arousal_array_raw, dominance_array_raw
  else:
    print('Opening raw data. Might take a while')
    sc_array_raw = np.empty(shape=(subject_count*trial_count_per_subject, signal_length))
    valence_array_raw = np.empty(shape=(subject_count*trial_count_per_subject))
    arousal_array_raw = np.empty(shape=(subject_count*trial_count_per_subject))
    dominance_array_raw = np.empty(shape=(subject_count*trial_count_per_subject))
    counter = 0
    for subject in range(1, subject_count+1):
      print('Opening File of Subject ', subject, '...')
      if subject < 10:
          subject_raw_data_dir = DEAP_data_directory + "/raw_data/s0{subject_number}.dat".format(subject_number=subject)
      else:
          subject_raw_data_dir = DEAP_data_directory + "/raw_data/s{subject_number}.dat".format(subject_number=subject)
      with open(subject_raw_data_dir, 'rb') as f:
          d = pickle.load(f, encoding='latin1')
          for trial in range(trial_count_per_subject):
              sc_array_raw[counter] =d['data'][trial][sc_index]
              valence_array_raw[counter] = d['labels'][trial][0]
              arousal_array_raw[counter] = d['labels'][trial][1]
              dominance_array_raw[counter] = d['labels'][trial][2]
              counter += 1
                     
    with open(DEAP_data_directory + '/sc_array_raw', 'wb') as f:
      pickle.dump(sc_array_raw, f, protocol=pickle.HIGHEST_PROTOCOL)  
    with open(DEAP_data_directory + '/valence_array_raw', 'wb') as f:
      pickle.dump(valence_array_raw, f, protocol=pickle.HIGHEST_PROTOCOL)  
    with open(DEAP_data_directory + '/arousal_array_raw', 'wb') as f:
      pickle.dump(arousal_array_raw, f, protocol=pickle.HIGHEST_PROTOCOL)  
    with open(DEAP_data_directory + '/dominance_array_raw', 'wb') as f:
      pickle.dump(dominance_array_raw, f, protocol=pickle.HIGHEST_PROTOCOL)  

    return sc_array_raw, valence_array_raw, arousal_array_raw, dominance_array_raw  

Now, let's make a function that processes the SC data to be representative of a wearable device. 

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, resample

def filter_sc_signal_noise(sc_signal, cutoff_frequency, fs=None):
    if fs is None:
        fs = 128
    cutoff = cutoff_frequency
    order = 2
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    sc_signal_filtered = filtfilt(b, a, sc_signal)
    return sc_signal_filtered


def subsample(signal, desired_frequency, cutoff_begin, cutoff_end):
    signal = resample(signal, num=63*desired_frequency)
    return signal[cutoff_begin*desired_frequency:-cutoff_end*desired_frequency]

def process_data(sc_array_raw, cutoff_frequency, desired_sample_frequency):
  sc_array_processed = np.empty(shape=(len(sc_array_raw), 60*desired_sample_frequency))
  # First, let' filter and subsample the signal
  for signal_no in range(len(sc_array_raw)):
    signal = sc_array_raw[signal_no]
    signal = filter_sc_signal_noise(sc_signal=signal, cutoff_frequency=cutoff_frequency)
    signal = subsample(signal=signal, desired_frequency=desired_sample_frequency, cutoff_begin=2, cutoff_end=1)
    sc_array_processed[signal_no] = signal
  # Now, lets get the maximum SC value per subject and use that to normalize the signal
  max_sc_list = []
  for signal_number in range(0, len(sc_array_raw), trial_count_per_subject):
    signals = sc_array_processed[signal_number:(signal_number+trial_count_per_subject)]
    sc_max = np.amax(signals)
    max_sc_list.append(sc_max)
    sc_array_processed[signal_number:(signal_number+trial_count_per_subject)] = sc_array_processed[signal_number:(signal_number+40)]/sc_max

  return sc_array_processed

Now, lets get the Tonic and Phasic Components From The Signals:

In [ ]:
"""
______________________________________________________________________________
 File:                         cvxEDA.py
 Last revised:                 07 Nov 2015 r69
 ______________________________________________________________________________
 Copyright (C) 2014-2015 Luca Citi, Alberto Greco
 
 This program is free software; you can redistribute it and/or modify it under
 the terms of the GNU General Public License as published by the Free Software
 Foundation; either version 3 of the License, or (at your option) any later
 version.
 
 This program is distributed in the hope that it will be useful, but WITHOUT
 ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
 FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
 
 You may contact the author by e-mail (lciti@ieee.org).
 ______________________________________________________________________________
 This method was first proposed in:
 A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi
 "cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing"
 IEEE Transactions on Biomedical Engineering, 2015
 DOI: 10.1109/TBME.2015.2474131
 If you use this program in support of published research, please include a
 citation of the reference above. If you use this code in a software package,
 please explicitly inform the end users of this copyright notice and ask them
 to cite the reference above in their published research.
 ______________________________________________________________________________
"""

import cvxopt as cv
import cvxopt.solvers

def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """CVXEDA Convex optimization approach to electrodermal activity processing
    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).
    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters
    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C], 
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return (np.array(a).ravel() for a in (r, p, t, l, d, e, obj))
  


In [ ]:
def apply_cvx(sc_array_processed):
  phasic_sc_array = np.empty(shape=(1280, 60*desired_sample_frequency))
  tonic_sc_array = np.empty(shape=(1280, 60*desired_sample_frequency))

  for i in range(len(sc_array_processed)):
    [r, p, t, l, d, e, obj] = cvxEDA(sc_array_processed[i], 1./desired_sample_frequency)
    phasic_sc_array[i] = r
    tonic_sc_array[i] = t
  return phasic_sc_array, tonic_sc_array


Now, let's build a set of features for each type:

1. sc_array_processed
2. phasic_sc_array
3. tonic_sc_array

In [ ]:
!pip install tsfresh

In [ ]:
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
import pandas as pd

def prep_array_for_feature_extraction(input_array):
  return_df = pd.DataFrame(input_array)
  return_df = return_df.stack()
  return_df.index.rename([ 'id', 'time' ], inplace = True )
  return_df = return_df.reset_index()
  return return_df

def extract_all_features(sc_array_processed, phasic_sc_array, tonic_sc_array):
  proccessed_features = extract_features(prep_array_for_feature_extraction(sc_array_processed), column_id="id", column_sort="time")
  phasic_features = extract_features(prep_array_for_feature_extraction(phasic_sc_array), column_id="id", column_sort="time")
  tonic_features = extract_features(prep_array_for_feature_extraction(tonic_sc_array), column_id="id", column_sort="time")
  proccessed_features = impute(proccessed_features)
  phasic_features = impute(phasic_features)
  tonic_features = impute(tonic_features)
  return proccessed_features, phasic_features, tonic_features

def get_features(sc_array_processed, phasic_sc_array, tonic_sc_array):
  proccessed_features, phasic_features, tonic_features = extract_all_features(sc_array_processed, phasic_sc_array, tonic_sc_array)
  return proccessed_features, phasic_features, tonic_features


Now, let's process the label values. 

Since we are classifying two classes, let's use a boolean representation for label counts. True represents high, False represents low. 

In [ ]:
def process_label(label_value):
    if label_value > 5.5:
        return True
    else:
      return False

def get_labels():
  valence_processed = np.empty(shape=(1280))
  arousal_processed = np.empty(shape=(1280))
  dominance_processed = np.empty(shape=(1280))
  for i in range(len(valence_array_raw)):
    valence_processed[i] = process_label(valence_array_raw[i])
    arousal_processed[i] = process_label(arousal_array_raw[i])
    dominance_processed[i] = process_label(dominance_array_raw[i])
  print("Class imbalance (Full):")
  print("Valence levels: ", np.unique(valence_processed, return_counts=True))
  print("Arousal levels: ", np.unique(arousal_processed, return_counts=True))
  print("Dominance levels: ", np.unique(dominance_processed, return_counts=True))
  return valence_processed, arousal_processed, dominance_processed

Fit the extracted features to label values and store the resulting features in a format convenient for the classification tasks

In [ ]:
def get_selected_features(extracted_features, extracted_labels):
  selected_feat_dict = {}
  #First, let's combinte the features. 
  combined_features = pd.concat([extracted_features['processed_features'].add_prefix('o_'),
                      extracted_features['phasic_features'].add_prefix('p_'),
                      extracted_features['tonic_features'].add_prefix('t_')], axis=1)
  
  extracted_features['combined'] = combined_features

  # Now, selected features with the tsfresh package
  for feat_df_name, feat_df in extracted_features.items():
    for label_array_name, label_array in extracted_labels.items():
      selected_features = select_features(feat_df, pd.Series(label_array))
      if len(selected_features.columns) > 0:
        selected_feat_dict[feat_df_name + '_SELECTED-' + label_array_name] = {'features': selected_features,
                                                                    'labels': label_array,
                                                                    'features_count': len(selected_features.columns)}


  return selected_feat_dict

Now, let's build the K-fold routine to test a bunch of classification algos on each of the feature-label combos in the selected_feat_dict.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.dummy import DummyClassifier


def evaluate_experiments(selected_feat_dict):
  classifiers = {'Dummy': DummyClassifier(),
                'Random Forest': RandomForestClassifier(),
                'Logistic Regression': LogisticRegression(max_iter=1000),
                'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
                'KNeighborsClassifier': KNeighborsClassifier(),
                'GaussianNB': GaussianNB(),
                'DecisionTree': DecisionTreeClassifier(),
                'SVM': SVC(),
                }
  results_dict = {}
  skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=9) 
  for feat_label_name, data_dict in selected_feat_dict.items():
    X = data_dict['features']
    X = X.to_numpy()
    X = preprocessing.scale(X)
    Y = data_dict['labels'] 
    for classifier_name, classifier_model in classifiers.items(): 
      accuracy_list = []
      f1_list = []
      random_guess_list = []
      for train, test in skf.split(X, Y):
        classifier = classifier_model
        classifier.fit(X[train], Y[train])
        pred = classifier.predict(X[test])
        results = classification_report(Y[test], pred, output_dict=True)
        accuracy_list.append(results['accuracy'])
        f1_list.append(results['weighted avg']['f1-score'])
      results_dict[feat_label_name + ' | ' + classifier_name] = {'Mean Accuracy': np.mean(accuracy_list),
                                                                 'Accuracy STD': np.std(accuracy_list),
                                                                 'Mean F1 Score': np.mean(f1_list),
                                                                 'F1 Score STD': np.std(f1_list)}

  return(results_dict)

Experiment execution block (modify this)

In [ ]:
import os
import pandas as pd
import sys
sys.stdout = sys.__stdout__ # Unsilence the print statements. 

# First, let's define the hyper parameters:
desired_sample_frequency = 10 #Hz, this probably will remain constant
desired_signal_length_seconds = 10 # Last N second that we want to use for classification
desired_signal_length = desired_signal_length_seconds * desired_sample_frequency # Samples in each signal
cutoff_frequency = 1 # When filtering the raw SC signal, we choose the cutoff frequency for the lowpass filter. 

# Now, our folder directories 
DEAP_data_directory = "/content/drive/My Drive/Engineering/Datasets/DEAP"
features_directory = DEAP_data_directory + "/sc/features"
results_directory = DEAP_data_directory + "/sc/results"

# Now, file names
name_ending = "_{length}s_{cutoff}hz.csv".format(length=desired_signal_length_seconds, cutoff=str(cutoff_frequency).replace('.', ''))
processed_features_name = "/processed_features" + name_ending
phasic_features_name = "/phasic_features" + name_ending
tonic_features_name = "/tonic_features" + name_ending
results_file_name = "/classic_ML_results" + name_ending

# Features files
processed_features_file = features_directory + processed_features_name
phasic_features_file = features_directory + phasic_features_name
tonic_features_file = features_directory + tonic_features_name
files = [processed_features_file, phasic_features_file, tonic_features_file]

# Results file:
results_file = results_directory + results_file_name

# Let's check if the the feature file for the signal-length & frequency combo 
# exists, and use that instead of going through the processing routine
files_exist = []
for file in files:
  if os.path.isfile(file):
    files_exist.append(True)
  else:
    files_exist.append(False)

if all(files_exist):
  # Use these files
  print("Freatures files for the signal-frequency combination exists. Using them...")
  proccessed_features = pd.read_csv(processed_features_file)
  phasic_features = pd.read_csv(phasic_features_file)
  tonic_features = pd.read_csv(tonic_features_file)
else: 
  # Initiate feature extraction
  # Step 0: Get raw data:
  sc_array_raw, valence_array_raw, arousal_array_raw, dominance_array_raw = get_raw_data(DEAP_data_directory) 
  # Step 1: Get the wearable data, decompose it to desred signal length
  # sc_array_processed
  # phasic_sc_array
  # tonic_sc_array
  print("Processing raw data into a wearable format...")
  sc_array_processed = process_data(sc_array_raw, cutoff_frequency=cutoff_frequency,
                                    desired_sample_frequency=desired_sample_frequency)
  
  print("Decomposing into phasic and tonic components using cvxEDA...")
  phasic_sc_array, tonic_sc_array = apply_cvx(sc_array_processed=sc_array_processed)

  # Step 3: Cut the signals to desired length
  print("Cutting the signals to the desired length")
  sc_data = np.empty(shape=(len(sc_array_processed), desired_signal_length))
  phasic_sc_data = np.empty(shape=(len(phasic_sc_array), desired_signal_length))
  tonic_sc_data = np.empty(shape=(len(tonic_sc_array), desired_signal_length))
  for i in range(len(sc_array_processed)):
    sc_data[i] = sc_array_processed[i][-desired_signal_length:]
    phasic_sc_data[i] = phasic_sc_array[i][-desired_signal_length:]
    tonic_sc_data[i] = tonic_sc_array[i][-desired_signal_length:]

  # Step 4: Get the features
  print("Extracting the features from processed, phasic and tonic signals.")
  processed_features, phasic_features, tonic_features = get_features(sc_array_processed=sc_data, 
                                                                    phasic_sc_array=phasic_sc_data, 
                                                                    tonic_sc_array=tonic_sc_data)
  
  # Step 5: Store the features into the files
  print("Storing the files with the ending: ", name_ending)
  processed_features.to_csv(processed_features_file)
  phasic_features.to_csv(phasic_features_file)
  tonic_features.to_csv(tonic_features_file)

# Now, let's prepare the labels
print("Preparing the labels")
valence_processed, arousal_processed, dominance_processed = get_labels()

# Select Features
print("Selecting the features using tsfresh")
extracted_features = {'processed_features': processed_features, 
                      'phasic_features': phasic_features, 
                      'tonic_features': tonic_features
                      }

extracted_labels = {'valence_processed': valence_processed,
                    'arousal_processed': arousal_processed,
                    'dominance_processed': dominance_processed
                    }

selected_feat_dict = get_selected_features(extracted_features=extracted_features,
                                           extracted_labels=extracted_labels)

# Test the classifiers and store the results
print("Testing the classifiers")
results_df = evaluate_experiments(selected_feat_dict=selected_feat_dict)
results_df = pd.DataFrame.from_dict(results_df)
results_df = results_df.transpose()
results_df.to_csv(results_file)